### Barrier Synchronization with Semaphores (Python)

Here is a solution to barrier synchronization that is symmetric in how each worker synchronizes; each worker first notifies the other worker of their arrival at the barrier and then waits for the other worker to arrive:

```
var barrier1, barrier2: semaphore = 0, 0
```
<div style="display:table">
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker1
    while true do
        task 1
        V(barrier1)
        P(barrier2)
```
  </div>
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker2
    while true do
        task 2
        V(barrier2)
        P(barrier1)
```
  </div>
</div>
<br>

To argue for correctness, two ghost variables, `c1` and `c2`, are introduced for the number of times each worker has arrived at their barrier. The barrier property states that `c1` and `c2` cannot differ by more than `1`:

```
BR: |c1 – c2| ≤ 1
```

The global invariant, `P`, includes the conditions that hold after each of the two `V` operations:

```algorithm
var barrier1, barrier2: semaphore = 0, 0
var c1, c2: integer = 0, 0
{P: BR ∧ (barrier1 > 0 ⇒ c1 = c2 ∨ c1 = c2 + 1) ∧ (barrier2 > 0 ⇒ c1 = c2 ∨ c1 + 1 = c2)}
```
<div style="display:table">
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker1
    {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
    while true do
        task 1
        ⟨barrier1, c1 := barrier1 + 1, c1 + 1⟩ 
        {P ∧ (c1 = c2 ∨ c1 = c2 + 1)}
        ⟨await barrier2 > 0 then
            barrier2 := barrier2 – 1⟩
        {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
```
  </div>
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker2
    {P ∧ (c1 = c2 ∨ c1 = c2 + 1)}
    while true do
        task 2
        ⟨barrier2, c2 := barrier2 + 1, c2 + 1⟩ 
        {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
        ⟨await barrier1 > 0 then
            barrier1 := barrier1 – 1⟩
        {P ∧ (c1 = c2 ∨ c1 = c2 + 1)}
```
  </div>
</div>

Here is an asymmetric scheme for barrier synchronization with semaphores for two processes, with locations marked by `🄰` and `🄱`:

```
var barrier1, barrier2: semaphore = 0, 0
```
<div style="display:table">
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker1
    while true do
        task 1
        P(barrier2) 🄰
        V(barrier1) 🄱
```
  </div>
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker2
    while true do
        task 2
        V(barrier2) 🄰
        P(barrier1) 🄱
```
  </div>
</div>
<br>

For `worker1` to be at `🄰`, `worker2` must have passed `🄰`. For `worker2` to be at `🄱`, `worker1` must have passed `🄱`. Hence, for a process to leave its barrier, the other must have also arrived at its barrier. To show this formally, two ghost variables, `c1` and `c2`, are introduced with global invariant `P`:

```algorithm
var barrier1, barrier2: semaphore = 0, 0
var c1, c2: integer = 0, 0
{P: BR ∧ (barrier1 > 0 ⇒ c1 = c2) ∧ (barrier2 > 0 ⇒ c1 + 1 = c2)}
```
<div style="display:table">
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker1
    {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
    while true do
        task 1
        ⟨await barrier2 > 0 then
            barrier2, c1 := barrier2 – 1, c1 + 1⟩
        {P ∧ c1 = c2}
        ⟨barrier1, c1 := barrier1 + 1, c1 + 1⟩
        {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
```
  </div>
  <div style = "display:table-cell; border-left:2em solid transparent">

```algorithm
process worker2
    {P ∧ c1 = c2}
    while true do
        task 2
        {P ∧ c1 = c2}
        ⟨barrier2, c2 := barrier2 + 1, c2 + 1⟩
        {P ∧ (c1 = c2 ∨ c1 + 1 = c2)}
        ⟨await barrier1 > 0 then barrier1 := barrier1 – 1⟩
        {P ∧ c1 = c2}
```
  </div>
</div>
<br>

Note that `BR` can, in the asymmetric version, be strengthened to `0 ≤ c2 – c1 ≤ 1`.

Generalize barrier synchronization to three processes!

```algorithm
YOUR ANSWER HERE
```

Here is the corresponding implementation in Python. Modify that to three processes as well!

In [ ]:
# With 2 workers
from threading import Thread, Semaphore
from time import sleep
from sys import stdout

class Ping(Thread):
    def run(self):
        while True:
            stdout.write('ping\n'); sleep(2)    # task
            barrier2.acquire()                  # wait
            barrier1.release()                  # signal


class Pong(Thread):
    def run(self):
        while True:
            stdout.write('pong\n'); sleep(4)    # task
            barrier2.release()                  # signal
            barrier1.acquire()                  # wait

barrier1, barrier2 = Semaphore(0), Semaphore(0) # create semaphores
Ping().start(); Pong().start()                  # create and run threads

In [ ]:

YOUR ANSWER HERE

Output of `ping` and `pong` may not appear in the order in which they were written due to buffering. Use `stdout.flush()` to force buffers to be flushed. Will that always guarantee output in the same order in which `stdout.write()` occurred? Type `help(type(stdout))` in a code cell to see the interval in which buffers are flushed in the underlying implementation (you are not supposed to make use of that interval, as it may change).

YOUR ANSWER HERE